In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_json ('caselaw_extraction/benchmark.json')
df.head()

,filename,rule_id,citation
0,2012-ewhc-90219-costs.xml,ewhc_ch,[2007] EWHC 2733 (Ch)
1,2012-ewhc-90219-costs.xml,ewhc_qb,[2008] EWHC 497 (QB)
2,2012-ewhc-90219-costs.xml,ewca_civ,[2003] EWCA Civ 1766
3,2012-ewhc-90219-costs.xml,ewca_civ,[2005] EWCA Civ 1206
4,2012-ewhc-90219-costs.xml,ewhc_qb,[2003] EWHC 3127 (QB)


In [65]:
df_vlex = pd.read_csv('tna-sample/vcite-sample-caselaw-analysis.csv')
df_vlex.Filename = df_vlex.Filename.str.split('/').apply(lambda x: x[-1])
df_vlex = df_vlex[df_vlex.Filename!='2021-ewhc-65-fam.xml']
df_vlex = df_vlex[['Filename', 'Content', 'Error', 'ExtractedSeries']]
df_vlex.head()

len(df_vlex)

12355

In [66]:
df_vlex = df_vlex[~df_vlex.Content.isin(['HCA', 'NZLR', 'N.Z.L.R', 'N.Y.', 'U.S.', 'US', 'A.D.', 'F. Supp.', 'NSWCA', 'NSWLR', 'NSWSC', 'CLC', 'CLR', 'SASR',
'1F487', 'say', 'Jenkins', 'Cal.'])]
len(df_vlex)

12312

In [26]:
our_files = df.filename.unique().tolist()
vlex_files = df_vlex.Filename.unique().tolist()

print(len(our_files))
print(len(vlex_files))

388
389


In [27]:
list(set(vlex_files)-set(our_files))

['2006-ewhc-1187-tcc.xml',
 '2021-ewhc-794-tcc.xml',
 '2009-ewhc-1274-admlty.xml',
 '2019-ewca-1402-civ.xml']

In [28]:
list(set(our_files)-set(vlex_files))

['2021-ewhc-5-costs.xml', '2021-ukpc-24.xml', '2014-ewhc-1195-mercantile.xml']

In [67]:
comparison_dict = {'filename': [], 'num_both_extracted': [], 'num_mdr_extracted': [], 'num_vlex_extracted': [], 'num_mdr_not_extracted': [], 'num_vlex_not_extracted': []}

for i in vlex_files:
  our_df = df[df.filename==i]
  vlex_df = df_vlex[df_vlex.Filename==i]

  comparison_df = our_df.merge(vlex_df, indicator=True, how='outer', left_on=['filename', 'citation'], right_on=['Filename', 'Content'])
  mdr_extracted = len(comparison_df[comparison_df._merge=='both'])+len(comparison_df[comparison_df._merge=='left_only'])
  vlex_extracted = len(comparison_df[comparison_df._merge=='both'])+len(comparison_df[comparison_df._merge=='right_only'])
  same_extraction = len(comparison_df[comparison_df._merge=='both'])
  mdr_not_extracted = len(comparison_df[comparison_df._merge=='right_only'])
  vlex_not_extracted = len(comparison_df[comparison_df._merge=='left_only'])

  comparison_dict['filename'].append(i)
  comparison_dict['num_both_extracted'].append(same_extraction)
  comparison_dict['num_mdr_extracted'].append(mdr_extracted)
  comparison_dict['num_vlex_extracted'].append(vlex_extracted)
  comparison_dict['num_mdr_not_extracted'].append(mdr_not_extracted)
  comparison_dict['num_vlex_not_extracted'].append(vlex_not_extracted)

In [68]:
outcome = pd.DataFrame.from_dict(comparison_dict)
outcome['benchmark_percent'] = (outcome.num_mdr_extracted)/(outcome.num_vlex_extracted)
outcome.head(20)

,filename,num_both_extracted,num_mdr_extracted,num_vlex_extracted,num_mdr_not_extracted,num_vlex_not_extracted,benchmark_percent
0,2012-ewhc-90219-costs.xml,6,7,6,0,1,1.166667
1,2012-ewhc-90223-costs.xml,6,6,11,5,0,0.545455
2,2008-ewhc-90111-costs.xml,12,14,12,0,2,1.166667
3,2008-ewhc-90105-costs.xml,16,19,18,2,3,1.055556
4,2008-ewhc-90110-costs.xml,29,30,37,8,1,0.810811
5,2008-ewhc-90107-costs.xml,7,7,13,6,0,0.538462
6,2009-ewhc-90154-costs.xml,9,12,9,0,3,1.333333
7,2009-ewhc-90144-costs.xml,5,6,6,1,1,1.000000
8,2009-ewhc-90133-costs.xml,5,5,5,0,0,1.000000
9,2011-ewhc-90208-costs.xml,13,13,13,0,0,1.000000


In [69]:
print(outcome.num_mdr_extracted.sum())
print(outcome.num_vlex_extracted.sum())

18352
18421


In [70]:
outcome.benchmark_percent.mean()

0.9805035132307487

In [71]:
len(outcome[outcome.num_mdr_extracted > outcome.num_vlex_extracted])

160

In [72]:
len(outcome[outcome.num_mdr_extracted < outcome.num_vlex_extracted])

172

In [35]:
merged = example_df.merge(example_df_vlex, indicator=True, how='outer', left_on='citation', right_on='Content')
#merged[merged['_merge'] == 'left_only']

In [36]:
merged

,filename,rule_id,citation,Filename,Original,RuleName,CitationType,MatchType,Vids,Nodes,Content,Error,Notes,ExtractedSeries,_merge
0,2008-ewhc-90117-costs.xml,lrqb_single,[1998] QB 781,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,"style=""font-family:'Times New Roman';font-size...",elr3,Caselaw,SyntalexMatch,793741117,NaN,[1998] QB 781,NaN,Resolved match,QB,both
1,2008-ewhc-90117-costs.xml,ewca_civ,[2002] EWCA Civ 932,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,"style=""font-family:'Times New Roman';font-size...",EW Civil Division,Caselaw,SyntalexMatch,793571245,NaN,[2002] EWCA Civ 932,NaN,Resolved match,EWCA Civ,both
2,2008-ewhc-90117-costs.xml,lrqb_single,[2003] QB 381,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,",<citation entityType='SyntalexMatch' ruleName...",elr3,Caselaw,SyntalexMatch,793571245,NaN,[2003] QB 381,NaN,Resolved match,QB,both
3,2008-ewhc-90117-costs.xml,lrqb_single,[2003] QB 381,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,"style=""font-family:'Times New Roman';font-size...",elr3,Caselaw,SyntalexMatch,793571245,NaN,[2003] QB 381,NaN,Resolved match,QB,both
4,2008-ewhc-90117-costs.xml,lrqb_single,[2003] QB 381,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,",<citation entityType='SyntalexMatch' ruleName...",elr3,Caselaw,SyntalexMatch,793571245,NaN,[2003] QB 381,NaN,Resolved match,QB,both
5,2008-ewhc-90117-costs.xml,lrqb_single,[2003] QB 381,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,"style=""font-family:'Times New Roman';font-size...",elr3,Caselaw,SyntalexMatch,793571245,NaN,[2003] QB 381,NaN,Resolved match,QB,both
6,2008-ewhc-90117-costs.xml,lrac_multi,[1994] 1 AC 142,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,"style=""font-family:'Times New Roman';font-size...",elr3,Caselaw,SyntalexMatch,792966517,NaN,[1994] 1 AC 142,NaN,Resolved match,AC,both
7,2008-ewhc-90117-costs.xml,lrac_multi,[1994] 1 AC 142,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,"style=""font-family:'Times New Roman';font-size...",elr3,Caselaw,SyntalexMatch,792966517,NaN,[1994] 1 AC 142,NaN,Resolved match,AC,both
8,2008-ewhc-90117-costs.xml,lrac_multi,[1994] 1 AC 142,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,"style=""font-family:'Times New Roman';font-size...",elr3,Caselaw,SyntalexMatch,792966517,NaN,[1994] 1 AC 142,NaN,Resolved match,AC,both
9,2008-ewhc-90117-costs.xml,lrac_multi,[1994] 1 AC 142,/ewhc/costs/2008/2008-ewhc-90117-costs.xml,"style=""font-family:'Times New Roman';font-size...",elr3,Caselaw,SyntalexMatch,792966517,NaN,[1994] 1 AC 142,NaN,Resolved match,AC,both
